In [26]:
#basics
import random
import pandas as pd
import torch
#extra:
import os
import nltk
from lxml import etree

In [63]:
#sentence_id : id for the sentence, can be found in the xml data
#token_id: id for the word in the vocabulary
#char_start_id: where the character starts in the sentence, can be found in the xml data
#char_end_id: where the character ends in the sentence, can be found in the xml data
#split: which data split the token belong to, e.g. TRAIN, VAL or TEST.
#ner_id: id of the NER label, e.g. if we have 3 labels there would be 3 ids.from glob import glob

def parse_data(data_dir):     
    vocab = [] #keeping track of unique words in the data
    data_dir = glob("{}/*".format(data_dir)) #glob returns a possibly-empty list of path names that match data_dir 
                                            #...in this case a list with the two subdirectories 'Test' and 'Train'                                          
    for subdir in data_dir: #looping through 'Test' and 'Train'
        split = os.path.basename(subdir) #get the directory name without path
        subdir = glob("{}/*".format(subdir))
        for folder in subdir:  #looping through 'Test for DDI Extraction task' and 'Test for DrugNER task'
            folder = glob("{}/*".format(folder))
            for subfolder in folder: #looping through 'DrugBank' and 'MedLine'
                subfolder = glob("{}/*".format(subfolder))
                for xml_file in subfolder:
                    parse_xml(xml_file, split, vocab)
                    print("##################################################################")
                        
def parse_xml(xml_file, split, vocab):    
    tree = etree.parse(xml_file)
    root = tree.getroot()
    
    for elem in root: #loop over sentence tags
        if elem.tag == 'sentence':
            sent_id = elem.attrib['id'] #get sentence id
            text = elem.attrib['text']  #get the sentence as a string of text
            text = text.replace('-', ' ') #replaces all hyphens with whitespace for easier split of compound words
            char_pos = -1 #variable for keeping track of character-based positions of the words in the sentence
            nltk_tokens = nltk.word_tokenize(text)
            for token in nltk_tokens:
            #for word in text.split():
                char_pos += 1
                char_start = char_pos
                char_end = char_start + len(token)-1
                token_id, vocab = map_token_to_id(token, vocab)
                data_instance = [sent_id, token, token_id, char_start, char_end, split]
                print(data_instance)
                char_pos=char_end+1 #increase by 1 to account for the whitespace between the current and the next word 
    #for subelem in elem:
    #if subelem.tag == 'entity'
        #print('elem.attrib[text]: ', text)
        #for subelem in elem:                
    #return data_instance, vocabulary
    
def map_token_to_id(token, vocab):
    vocab = vocab
    if token not in vocab:
        vocab.append(token)
    token_id = vocab.index(token)
    return token_id, vocab

#def get_train_val_test_tag()
    
    
# one specific item's data
#print('\nItem #2 data:')
#print(root[0][1].text)

In [64]:
parse_data('/home/guserbto@GU.GU.SE/lt2316-h20-aa/DDICorpus')

['DDI-DrugBank.d610.s0', 'Pharmacokinetic', 0, 0, 14, 'Test']
['DDI-DrugBank.d610.s0', 'properties', 1, 16, 25, 'Test']
['DDI-DrugBank.d610.s0', 'of', 2, 27, 28, 'Test']
['DDI-DrugBank.d610.s0', 'abacavir', 3, 30, 37, 'Test']
['DDI-DrugBank.d610.s0', 'were', 4, 39, 42, 'Test']
['DDI-DrugBank.d610.s0', 'not', 5, 44, 46, 'Test']
['DDI-DrugBank.d610.s0', 'altered', 6, 48, 54, 'Test']
['DDI-DrugBank.d610.s0', 'by', 7, 56, 57, 'Test']
['DDI-DrugBank.d610.s0', 'the', 8, 59, 61, 'Test']
['DDI-DrugBank.d610.s0', 'addition', 9, 63, 70, 'Test']
['DDI-DrugBank.d610.s0', 'of', 2, 72, 73, 'Test']
['DDI-DrugBank.d610.s0', 'either', 10, 75, 80, 'Test']
['DDI-DrugBank.d610.s0', 'lamivudine', 11, 82, 91, 'Test']
['DDI-DrugBank.d610.s0', 'or', 12, 93, 94, 'Test']
['DDI-DrugBank.d610.s0', 'zidovudine', 13, 96, 105, 'Test']
['DDI-DrugBank.d610.s0', 'or', 12, 107, 108, 'Test']
['DDI-DrugBank.d610.s0', 'the', 8, 110, 112, 'Test']
['DDI-DrugBank.d610.s0', 'combination', 14, 114, 124, 'Test']
['DDI-DrugBank.d

['DDI-DrugBank.d776.s23', '400', 486, 44, 46, 'Test']
['DDI-DrugBank.d776.s23', 'mg', 57, 48, 49, 'Test']
['DDI-DrugBank.d776.s23', 'or', 12, 51, 52, 'Test']
['DDI-DrugBank.d776.s23', 'greater', 382, 54, 60, 'Test']
['DDI-DrugBank.d776.s23', 'with', 61, 62, 65, 'Test']
['DDI-DrugBank.d776.s23', 'terfenadine', 479, 67, 77, 'Test']
['DDI-DrugBank.d776.s23', 'is', 129, 79, 80, 'Test']
['DDI-DrugBank.d776.s23', 'contraindicated', 301, 82, 96, 'Test']
['DDI-DrugBank.d776.s23', '.', 16, 98, 98, 'Test']
['DDI-DrugBank.d776.s24', 'The', 43, 0, 2, 'Test']
['DDI-DrugBank.d776.s24', 'coadministration', 492, 4, 19, 'Test']
['DDI-DrugBank.d776.s24', 'of', 2, 21, 22, 'Test']
['DDI-DrugBank.d776.s24', 'fluconazole', 438, 24, 34, 'Test']
['DDI-DrugBank.d776.s24', 'at', 184, 36, 37, 'Test']
['DDI-DrugBank.d776.s24', 'doses', 185, 39, 43, 'Test']
['DDI-DrugBank.d776.s24', 'lower', 297, 45, 49, 'Test']
['DDI-DrugBank.d776.s24', 'than', 493, 51, 54, 'Test']
['DDI-DrugBank.d776.s24', '400', 486, 56, 58, 'T

['DDI-DrugBank.d597.s23', ':', 91, 13, 13, 'Test']
['DDI-DrugBank.d597.s23', 'A', 177, 15, 15, 'Test']
['DDI-DrugBank.d597.s23', 'study', 47, 17, 21, 'Test']
['DDI-DrugBank.d597.s23', 'in', 40, 23, 24, 'Test']
['DDI-DrugBank.d597.s23', '13', 809, 26, 27, 'Test']
['DDI-DrugBank.d597.s23', 'rheumatoid', 810, 29, 38, 'Test']
['DDI-DrugBank.d597.s23', 'arthritis', 811, 40, 48, 'Test']
['DDI-DrugBank.d597.s23', '(', 55, 50, 50, 'Test']
['DDI-DrugBank.d597.s23', 'RA', 812, 52, 53, 'Test']
['DDI-DrugBank.d597.s23', ')', 60, 55, 55, 'Test']
['DDI-DrugBank.d597.s23', 'patients', 51, 57, 64, 'Test']
['DDI-DrugBank.d597.s23', 'evaluated', 813, 66, 74, 'Test']
['DDI-DrugBank.d597.s23', 'the', 8, 76, 78, 'Test']
['DDI-DrugBank.d597.s23', 'effects', 270, 80, 86, 'Test']
['DDI-DrugBank.d597.s23', 'of', 2, 88, 89, 'Test']
['DDI-DrugBank.d597.s23', 'multiple', 162, 91, 98, 'Test']
['DDI-DrugBank.d597.s23', 'doses', 185, 100, 104, 'Test']
['DDI-DrugBank.d597.s23', 'of', 2, 106, 107, 'Test']
['DDI-DrugBa

['DDI-DrugBank.d779.s0', 'When', 303, 0, 3, 'Test']
['DDI-DrugBank.d779.s0', 'methyldopa', 1143, 5, 14, 'Test']
['DDI-DrugBank.d779.s0', 'is', 129, 16, 17, 'Test']
['DDI-DrugBank.d779.s0', 'used', 254, 19, 22, 'Test']
['DDI-DrugBank.d779.s0', 'with', 61, 24, 27, 'Test']
['DDI-DrugBank.d779.s0', 'other', 417, 29, 33, 'Test']
['DDI-DrugBank.d779.s0', 'antihypertensive', 639, 35, 50, 'Test']
['DDI-DrugBank.d779.s0', 'drugs', 509, 52, 56, 'Test']
['DDI-DrugBank.d779.s0', ',', 68, 58, 58, 'Test']
['DDI-DrugBank.d779.s0', 'potentiation', 1144, 60, 71, 'Test']
['DDI-DrugBank.d779.s0', 'of', 2, 73, 74, 'Test']
['DDI-DrugBank.d779.s0', 'antihypertensive', 639, 76, 91, 'Test']
['DDI-DrugBank.d779.s0', 'effect', 30, 93, 98, 'Test']
['DDI-DrugBank.d779.s0', 'may', 85, 100, 102, 'Test']
['DDI-DrugBank.d779.s0', 'occur', 296, 104, 108, 'Test']
['DDI-DrugBank.d779.s0', '.', 16, 110, 110, 'Test']
['DDI-DrugBank.d779.s1', 'Patients', 166, 0, 7, 'Test']
['DDI-DrugBank.d779.s1', 'should', 170, 9, 14, 'Te

['DDI-DrugBank.d588.s1', 'care', 1383, 91, 94, 'Test']
['DDI-DrugBank.d588.s1', 'must', 1384, 96, 99, 'Test']
['DDI-DrugBank.d588.s1', 'be', 86, 101, 102, 'Test']
['DDI-DrugBank.d588.s1', 'taken', 489, 104, 108, 'Test']
['DDI-DrugBank.d588.s1', ',', 68, 110, 110, 'Test']
['DDI-DrugBank.d588.s1', 'especially', 1385, 112, 121, 'Test']
['DDI-DrugBank.d588.s1', 'during', 255, 123, 128, 'Test']
['DDI-DrugBank.d588.s1', 'initial', 1021, 130, 136, 'Test']
['DDI-DrugBank.d588.s1', 'therapy', 54, 138, 144, 'Test']
['DDI-DrugBank.d588.s1', '.', 16, 146, 146, 'Test']
['DDI-DrugBank.d588.s2', 'Dosage', 1386, 0, 5, 'Test']
['DDI-DrugBank.d588.s2', 'adjustments', 1277, 7, 17, 'Test']
['DDI-DrugBank.d588.s2', 'of', 2, 19, 20, 'Test']
['DDI-DrugBank.d588.s2', 'other', 417, 22, 26, 'Test']
['DDI-DrugBank.d588.s2', 'antihypertensives', 1387, 28, 44, 'Test']
['DDI-DrugBank.d588.s2', 'may', 85, 46, 48, 'Test']
['DDI-DrugBank.d588.s2', 'be', 86, 50, 51, 'Test']
['DDI-DrugBank.d588.s2', 'necessary', 422, 53

...

['DDI-MedLine.d171.s5', 'nucleoside', 5560, 39, 48, 'Test']
['DDI-MedLine.d171.s5', 'or', 12, 50, 51, 'Test']
['DDI-MedLine.d171.s5', 'nucleotide', 5561, 53, 62, 'Test']
['DDI-MedLine.d171.s5', 'reverse', 1241, 64, 70, 'Test']
['DDI-MedLine.d171.s5', 'transcriptase', 1242, 72, 84, 'Test']
['DDI-MedLine.d171.s5', 'inhibitors', 144, 86, 95, 'Test']
['DDI-MedLine.d171.s5', '(', 55, 97, 97, 'Test']
['DDI-MedLine.d171.s5', 'NRTI', 5562, 99, 102, 'Test']
['DDI-MedLine.d171.s5', ')', 60, 104, 104, 'Test']
['DDI-MedLine.d171.s5', 'together', 856, 106, 113, 'Test']
['DDI-MedLine.d171.s5', 'with', 61, 115, 118, 'Test']
['DDI-MedLine.d171.s5', 'a', 46, 120, 120, 'Test']
['DDI-MedLine.d171.s5', 'non', 1870, 122, 124, 'Test']
['DDI-MedLine.d171.s5', 'nucleoside', 5560, 126, 135, 'Test']
['DDI-MedLine.d171.s5', 'reverse', 1241, 137, 143, 'Test']
['DDI-MedLine.d171.s5', 'transcriptase', 1242, 145, 157, 'Test']
['DDI-MedLine.d171.s5', 'inhibitor', 304, 159, 167, 'Test']
['DDI-MedLine.d171.s5', '(', 55

['DDI-MedLine.d191.s0', '[', 3184, 0, 0, 'Test']
['DDI-MedLine.d191.s0', 'Study', 4525, 2, 6, 'Test']
['DDI-MedLine.d191.s0', 'on', 31, 8, 9, 'Test']
['DDI-MedLine.d191.s0', 'synergic', 5693, 11, 18, 'Test']
['DDI-MedLine.d191.s0', 'and', 15, 20, 22, 'Test']
['DDI-MedLine.d191.s0', 'decreasing', 538, 24, 33, 'Test']
['DDI-MedLine.d191.s0', 'toxic', 1119, 35, 39, 'Test']
['DDI-MedLine.d191.s0', 'effects', 270, 41, 47, 'Test']
['DDI-MedLine.d191.s0', 'of', 2, 49, 50, 'Test']
['DDI-MedLine.d191.s0', 'mineral', 4500, 52, 58, 'Test']
['DDI-MedLine.d191.s0', 'water', 1400, 60, 64, 'Test']
['DDI-MedLine.d191.s0', 'and', 15, 66, 68, 'Test']
['DDI-MedLine.d191.s0', 'Chinese', 3788, 70, 76, 'Test']
['DDI-MedLine.d191.s0', 'herbal', 2997, 78, 83, 'Test']
['DDI-MedLine.d191.s0', 'compound', 3482, 85, 92, 'Test']
['DDI-MedLine.d191.s0', 'preparation', 3811, 94, 104, 'Test']
['DDI-MedLine.d191.s0', 'on', 31, 106, 107, 'Test']
['DDI-MedLine.d191.s0', 'cisplatin', 1805, 109, 117, 'Test']
['DDI-MedLine

['DDI-MedLine.d148.s6', ',', 68, 85, 85, 'Test']
['DDI-MedLine.d148.s6', 'and', 15, 87, 89, 'Test']
['DDI-MedLine.d148.s6', 'delineate', 5811, 91, 99, 'Test']
['DDI-MedLine.d148.s6', 'a', 46, 101, 101, 'Test']
['DDI-MedLine.d148.s6', 'new', 844, 103, 105, 'Test']
['DDI-MedLine.d148.s6', 'means', 3388, 107, 111, 'Test']
['DDI-MedLine.d148.s6', 'of', 2, 113, 114, 'Test']
['DDI-MedLine.d148.s6', 'communication', 5812, 116, 128, 'Test']
['DDI-MedLine.d148.s6', 'between', 365, 130, 136, 'Test']
['DDI-MedLine.d148.s6', 'nucleus', 5452, 138, 144, 'Test']
['DDI-MedLine.d148.s6', 'and', 15, 146, 148, 'Test']
['DDI-MedLine.d148.s6', 'synapse', 5813, 150, 156, 'Test']
['DDI-MedLine.d148.s6', 'that', 321, 158, 161, 'Test']
['DDI-MedLine.d148.s6', 'may', 85, 163, 165, 'Test']
['DDI-MedLine.d148.s6', 'provide', 3494, 167, 173, 'Test']
['DDI-MedLine.d148.s6', 'insight', 5254, 175, 181, 'Test']
['DDI-MedLine.d148.s6', 'into', 120, 183, 186, 'Test']
['DDI-MedLine.d148.s6', 'normal', 1531, 188, 193, 'Te

['DDI-MedLine.d220.s2', 'apigenin', 6017, 13, 20, 'Test']
['DDI-MedLine.d220.s2', 'experienced', 3710, 22, 32, 'Test']
['DDI-MedLine.d220.s2', 'extensive', 1484, 34, 42, 'Test']
['DDI-MedLine.d220.s2', 'first', 823, 44, 48, 'Test']
['DDI-MedLine.d220.s2', 'pass', 3635, 50, 53, 'Test']
['DDI-MedLine.d220.s2', 'metabolism', 412, 55, 64, 'Test']
['DDI-MedLine.d220.s2', '.', 16, 66, 66, 'Test']
['DDI-MedLine.d220.s3', 'The', 43, 0, 2, 'Test']
['DDI-MedLine.d220.s3', 'elimination', 36, 4, 14, 'Test']
['DDI-MedLine.d220.s3', 'percent', 6020, 16, 22, 'Test']
['DDI-MedLine.d220.s3', 'of', 2, 24, 25, 'Test']
['DDI-MedLine.d220.s3', 'luteolin', 6016, 27, 34, 'Test']
['DDI-MedLine.d220.s3', 'and', 15, 36, 38, 'Test']
['DDI-MedLine.d220.s3', 'apigenin', 6017, 40, 47, 'Test']
['DDI-MedLine.d220.s3', 'was', 148, 49, 51, 'Test']
['DDI-MedLine.d220.s3', 'found', 502, 53, 57, 'Test']
['DDI-MedLine.d220.s3', 'to', 21, 59, 60, 'Test']
['DDI-MedLine.d220.s3', 'be', 86, 62, 63, 'Test']
['DDI-MedLine.d220.s

['DDI-MedLine.d153.s1', 'determination', 3791, 155, 167, 'Test']
['DDI-MedLine.d153.s1', 'of', 2, 169, 170, 'Test']
['DDI-MedLine.d153.s1', 'propiverine', 6114, 172, 182, 'Test']
['DDI-MedLine.d153.s1', 'hydrochloride', 2104, 184, 196, 'Test']
['DDI-MedLine.d153.s1', '(', 55, 198, 198, 'Test']
['DDI-MedLine.d153.s1', 'CAS', 4224, 200, 202, 'Test']
['DDI-MedLine.d153.s1', '54556', 6117, 204, 208, 'Test']
['DDI-MedLine.d153.s1', '98', 5959, 210, 211, 'Test']
['DDI-MedLine.d153.s1', '8', 1483, 213, 213, 'Test']
['DDI-MedLine.d153.s1', ')', 60, 215, 215, 'Test']
['DDI-MedLine.d153.s1', 'in', 40, 217, 218, 'Test']
['DDI-MedLine.d153.s1', 'human', 194, 220, 224, 'Test']
['DDI-MedLine.d153.s1', 'plasma', 323, 226, 231, 'Test']
['DDI-MedLine.d153.s1', 'using', 1069, 233, 237, 'Test']
['DDI-MedLine.d153.s1', 'cetirizine', 6118, 239, 248, 'Test']
['DDI-MedLine.d153.s1', 'di', 6119, 250, 251, 'Test']
['DDI-MedLine.d153.s1', 'hydrochloride', 2104, 253, 265, 'Test']
['DDI-MedLine.d153.s1', 'as', 42

In [51]:
xml = '<a xmlns="test"><b xmlns="test"/></a>'
root = etree.fromstring(xml)
etree.tostring(root)

b'<a xmlns="test"><b xmlns="test"/></a>'